<a href="https://colab.research.google.com/github/sid-mercor/mercor-scicode/blob/main/Multiparticle_dynamics_subproblem_32_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np


In [58]:
def runge_kutta_dynamics(C0, H, L, M, t0, steps):
    """
    Simulate the evolution of the correlation matrix using 4th-order Runge-Kutta dynamics.
    """
    dt = t0 / steps  # Time step size
    C = C0.astype(complex)  # Ensure complex dtype for operations

    for step in range(steps):
        # Compute intermediate Runge-Kutta terms
        k1 = dt * (1j * (H @ C - C @ H) - np.multiply(L, C) + M)
        k2 = dt * (1j * (H @ (C + 0.5 * k1) - (C + 0.5 * k1) @ H) - np.multiply(L, (C + 0.5 * k1)) + M)
        k3 = dt * (1j * (H @ (C + 0.5 * k2) - (C + 0.5 * k2) @ H) - np.multiply(L, (C + 0.5 * k2)) + M)
        k4 = dt * (1j * (H @ (C + k3) - (C + k3) @ H) - np.multiply(L, (C + k3)) + M)

        # Update correlation matrix using Runge-Kutta rule
        C += (k1 + 2 * k2 + 2 * k3 + k4) / 6

    return C


In [50]:
def mock_hamiltonian(N):
    """
    Generate a mock Hamiltonian matrix for testing purposes.

    Parameters:
    N (int): Size of the Hamiltonian matrix.

    Returns:
    numpy.ndarray: Mock Hamiltonian matrix (complex).
    """
    H = np.zeros((N, N), dtype=np.complex128)
    for i in range(N):
        H[i, i] = 1.0
    for i in range(N - 1):
        H[i, i + 1] = H[i + 1, i] = -0.5
    return H


In [51]:
def mock_dissipative_operator(N):
    """
    Generate a mock dissipative operator matrix for testing purposes.

    Parameters:
    N (int): Size of the dissipative operator matrix.

    Returns:
    numpy.ndarray: Mock dissipative operator matrix (complex).
    """
    return np.eye(N, dtype=np.complex128) * 0.1


In [52]:
def mock_driving_matrix(N):
    """
    Generate a mock driving matrix for testing purposes.

    Parameters:
    N (int): Size of the driving matrix.

    Returns:
    numpy.ndarray: Mock driving matrix (complex).
    """
    return np.zeros((N, N), dtype=np.complex128)


In [55]:
def test_runge_kutta_dynamics():
    """
    Unit test for the `runge_kutta_dynamics` function.
    Includes multiple test cases with varying Hamiltonian, dissipative operator,
    driving matrix, and initial conditions.
    """
    # Test Case 1: Simplified case with no dynamics
    C0 = np.array([[1.0, 0.0], [0.0, 0.0]], dtype=complex)
    H = np.zeros((2, 2), dtype=complex)  # Zero Hamiltonian
    L = np.zeros((2, 2), dtype=complex)  # Zero dissipative operator
    M = np.zeros((2, 2), dtype=complex)  # Zero driving matrix
    t0 = 1.0
    steps = 100
    expected = C0
    result = runge_kutta_dynamics(C0, H, L, M, t0, steps)
    assert np.allclose(result, expected, rtol=1e-4, atol=1e-6), f"Test Case 1 Failed! Expected {expected}, got {result}."
    print("Test Case 1 Passed!")

    # Test Case 2: Simple Hamiltonian-driven dynamics
    C0 = np.array([[1.0, 0.0], [0.0, 0.0]], dtype=complex)
    H = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)  # Off-diagonal Hamiltonian
    L = np.zeros((2, 2), dtype=complex)  # Zero dissipative operator
    M = np.zeros((2, 2), dtype=complex)  # Zero driving matrix
    t0 = np.pi  # Evolution time corresponding to a quarter period
    steps = 100

    # Run the function
    result = runge_kutta_dynamics(C0, H, L, M, t0, steps)

    # Numerical validation using symmetry and trace conservation
    result_trace = result.trace()  # Compute trace of the result
    initial_trace = C0.trace()  # Compute trace of the initial state
    assert np.isclose(result_trace, initial_trace, atol=1e-6), (
        f"Trace mismatch! Expected {initial_trace}, got {result_trace}."
    )
    print("Test Case 2 Passed (Numerical Validation)!")

    # Test Case 3: Dissipative dynamics
    C0 = np.eye(2, dtype=complex)  # Initial correlation matrix
    H = np.zeros((2, 2), dtype=complex)  # No Hamiltonian dynamics
    L = np.array([[0.1, 0.0], [0.0, 0.2]], dtype=complex)  # Simple damping matrix
    M = np.zeros((2, 2), dtype=complex)  # No driving
    t0 = 1.0  # Total evolution time
    steps = 100

    # Run the function
    result = runge_kutta_dynamics(C0, H, L, M, t0, steps)

    # Validate against expected decay: exp(-t0 * L[i, i])
    expected = np.diag([np.exp(-t0 * L[0, 0]), np.exp(-t0 * L[1, 1])])

    # Compare result to expected values
    assert np.allclose(np.diag(result), np.diag(expected), rtol=1e-4, atol=1e-6), (
        f"Test Case 3 Failed! Expected {expected}, got {result}."
    )
    print("Test Case 3 Passed!")

    # Test Case 4: Driving matrix effects
    C0 = np.zeros((2, 2), dtype=complex)  # Initial zero matrix
    H = np.zeros((2, 2), dtype=complex)  # Zero Hamiltonian
    L = np.zeros((2, 2), dtype=complex)  # Zero dissipative operator
    M = np.array([[0.0, 0.1], [0.1, 0.0]], dtype=complex)  # Off-diagonal driving term
    t0 = 1.0
    steps = 100
    expected = M * t0  # Linear growth approximation for small t
    result = runge_kutta_dynamics(C0, H, L, M, t0, steps)
    assert np.allclose(result, expected, rtol=1e-4, atol=1e-6), f"Test Case 4 Failed! Expected {expected}, got {result}."
    print("Test Case 4 Passed!")

    # Test Case 5: Combined effects
    C0 = np.eye(2, dtype=complex) * 0.5  # Diagonal initial condition
    H = mock_hamiltonian(2)  # Mock Hamiltonian
    L = mock_dissipative_operator(2)  # Mock dissipative operator
    M = mock_driving_matrix(2)  # Mock driving matrix
    t0 = 2.0
    steps = 100
    result = runge_kutta_dynamics(C0, H, L, M, t0, steps)
    print("Test Case 5 Result:", result)
    print("Test Case 5 Passed! (Verification may require numerical analysis)")




In [57]:
if __name__ == "__main__":
    # Execute the test function
    test_runge_kutta_dynamics()
    print("All test cases passed for `runge_kutta_dynamics`!")

Test Case 1 Passed!
Test Case 2 Passed (Numerical Validation)!
Test Case 3 Passed!
Test Case 4 Passed!
Test Case 5 Result: [[0.40936538+0.j 0.        +0.j]
 [0.        +0.j 0.40936538+0.j]]
Test Case 5 Passed! (Verification may require numerical analysis)
All test cases passed for `runge_kutta_dynamics`!
